In [2]:
import SimpleITK as sitk
import DicomReader as Reader
import collections


In [2]:
import numpy as np

In [3]:
tumor_paths = r"SegmentationNo_0"
ablation_paths =r"SegmentationNo_1"

folder_path_plan = r"C:\Study_840\Series_9"
folder_path_validation = r"C:\Study_840\Series_10"
tumor_mask, tumor_reader = Reader.read_dcm_series(tumor_paths)
source_img_plan, img_plan_reader = Reader.read_dcm_series(folder_path_plan)
ablation_mask, ablation_reader = Reader.read_dcm_series(ablation_paths)
source_img_validation, img_validation_reader = Reader.read_dcm_series(folder_path_validation)



In [6]:
tuple_imgs = collections.namedtuple('tuple_imgs',
                                    ['img_plan',
                                     'img_validation',
                                     'ablation_mask',
                                     'tumor_mask'])
images = tuple_imgs(source_img_plan, source_img_validation, ablation_mask, tumor_mask)

In [8]:
sitk.Show(images.img_plan)

In [9]:
def resize_resample_images(images):
    
    tuple_resized_imgs = collections.namedtuple('tuple_resized_imgs',
                                                ['img_plan',
                                                 'img_validation',
                                                 'ablation_mask',
                                                 'tumor_mask'])
    dimension = images.img_plan.GetDimension()  #
    reference_spacing = np.ones(dimension) # resize to isotropic size
    reference_direction = np.identity(dimension).flatten()
    reference_size = [512] * dimension
    reference_origin = np.zeros(dimension)
    data = [images.img_plan, images.img_validation, images.ablation_mask, images.tumor_mask]
    #
    # reference_physical_size = np.zeros(dimension)
    # for img in data:
    #     reference_physical_size[:] = [(sz - 1) * spc if sz * spc > mx else mx for sz, spc, mx in
    #                                   zip(img.GetSize(), img.GetSpacing(), reference_physical_size)]
    # reference_spacing = [phys_sz / (sz - 1) for sz, phys_sz in zip(reference_size, reference_physical_size)]

    reference_image = sitk.Image(reference_size, images.img_plan.GetPixelIDValue())
    reference_image.SetOrigin(reference_origin)
    reference_image.SetSpacing(reference_spacing)
    reference_image.SetDirection(reference_direction)
    reference_center = np.array(
        reference_image.TransformContinuousIndexToPhysicalPoint(np.array(reference_image.GetSize()) / 2.0))

    #%% Paste the GT segmentation masks before transformation
    #tumor_mask_paste = (paste_roi_image(images.img_plan, images.tumor_mask))
    #ablation_mask_paste = (paste_roi_image(images.img_validation, images.ablation_mask))
    #images.tumor_mask = tumor_mask_paste
    #images.ablation_mask = ablation_mask_paste
#     print('pasted image mask direction', tumor_mask_paste.GetDirection())
#     print('pasted image mask origin', tumor_mask_paste.GetOrigin())
#     print('pasted image mask spacing', tumor_mask_paste.GetSpacing())
#     print('ABLATION:')
#     print('pasted image mask direction', ablation_mask_paste.GetDirection())
#     print('pasted image mask origin', ablation_mask_paste.GetOrigin())
#     print('pasted image mask spacing', ablation_mask_paste.GetSpacing())

    # %%  Apply transforms
    data_resized = []
    for idx,img in enumerate(data):
        transform = sitk.AffineTransform(dimension) # use affine transform with 3 dimensions
        transform.SetMatrix(img.GetDirection()) # set the cosine direction matrix
        # TODO: check translation when computing the segmentations
        transform.SetTranslation(np.array(img.GetOrigin()) - reference_origin) # set the translation.
        # Modify the transformation to align the centers of the original and reference image instead of their origins.
        centering_transform = sitk.TranslationTransform(dimension)
        img_center = np.array(img.TransformContinuousIndexToPhysicalPoint(np.array(img.GetSize()) / 2.0))
        centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - reference_center))
        centered_transform = sitk.Transform(transform)
        centered_transform.AddTransform(centering_transform)
        # Using the linear interpolator as these are intensity images, if there is a need to resample a ground truth
        # segmentation then the segmentation image should be resampled using the NearestNeighbor interpolator so that
        # no new labels are introduced.
        # inputImage-> TransformIndexToPhysicalPoint(inputImage->GetLargestPossibleRegion()->GetIndex()).
        # set all  output image parameters: origin, spacing, direction, starting index, and size.
        if (idx==1 or idx==2): # temporary solution to resample the GT image with NearestNeighbour
            resampled_img = sitk.Resample(img, reference_image, centered_transform, sitk.sitkNearestNeighbor, 0.0)

        else:
             resampled_img = sitk.Resample(img, reference_image, centered_transform, sitk.sitkLinear, 0.0)
        # append to list
        data_resized.append(resampled_img)
    print('TUMOR RESAMPLED:')
    print('resampled image mask direction', data_resized[0].GetDirection())
    print('resampled image mask origin', data_resized[0].GetOrigin())
    print('resampled image mask spacing', data_resized[0].GetSpacing())

    sitk.Show(data_resized[0], 'img1', debugOn=True)
    # assuming the order stays the same, reassigng back to tuple
    resized_imgs = tuple_resized_imgs(img_plan=data_resized[0],
                                      img_validation=data_resized[1],
                                      ablation_mask=data_resized[2],
                                      tumor_mask=data_resized[3])

In [7]:
images_resized = resize_resample_images(images)

TUMOR RESAMPLED:
resampled image mask direction (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
resampled image mask origin (0.0, 0.0, 0.0)
resampled image mask spacing (1.0, 1.0, 1.0)


In [1]:
sitk.Show(tumor_mask)

NameError: name 'sitk' is not defined